In [4]:
with open("input2.txt", "r") as file:
    text_lines = file.read().split("\n\n")


In [17]:
print(text_lines[1])
print("\n")
print(text_lines[0])

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}


px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}


In [16]:
processes: text_lines[0].split("\n")
parts = text_lines[1].split("\n")

In [15]:
def string_to_dict(string):
    string = string.strip("{}") 
    pairs = string.split(",")
    return {pair.split("=")[0]: int(pair.split("=")[1]) for pair in pairs}

result_dict = string_to_dict(parts[0])
print(result_dict)


{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}


In [33]:
def convert_string_to_set_of_dicts(input_string):
    lines = input_string.splitlines()
    result_set = []

    for line in lines:
        parts = line.split("{") 
        key = parts[0].strip()
        values = parts[1].strip()[:-1].split(",") 
        result_set.append({key: values})

    return result_set

# Example usage:
input_string = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}"""

result_set = convert_string_to_set_of_dicts(input_string)
print(result_set)


[{'px': ['a<2006:qkq', 'm>2090:A', 'rfg']}, {'pv': ['a>1716:R', 'A']}, {'lnx': ['m>1548:A', 'A']}, {'rfg': ['s<537:gd', 'x>2440:R', 'A']}, {'qs': ['s>3448:A', 'lnx']}, {'qkq': ['x<1416:A', 'crn']}, {'crn': ['x>2662:A', 'R']}, {'in': ['s<1351:px', 'qqz']}, {'qqz': ['s>2770:qs', 'm<1801:hdj', 'R']}, {'gd': ['a>3333:R', 'R']}, {'hdj': ['m>838:A', 'pv']}]


In [37]:
for i in result_set:
    print(i)

{'px': ['a<2006:qkq', 'm>2090:A', 'rfg']}
{'pv': ['a>1716:R', 'A']}
{'lnx': ['m>1548:A', 'A']}
{'rfg': ['s<537:gd', 'x>2440:R', 'A']}
{'qs': ['s>3448:A', 'lnx']}
{'qkq': ['x<1416:A', 'crn']}
{'crn': ['x>2662:A', 'R']}
{'in': ['s<1351:px', 'qqz']}
{'qqz': ['s>2770:qs', 'm<1801:hdj', 'R']}
{'gd': ['a>3333:R', 'R']}
{'hdj': ['m>838:A', 'pv']}


In [98]:
total = []
key_to_select = "pv"
selected_values = []
for d in result_set:
    if key_to_select in d:
        selected_values.append(d[key_to_select])
print(f'selected val: {selected_values[0]}')

for i in selected_values[0]:
    cond = i.split(":")
    for j in range(0, len(cond)):
        if '>' in cond[j]:
            temp1, temp2 = cond[j].split(">")
            if result_dict[temp1] > int(temp2):
                cond[j] = [result_dict[temp1], '>', int(temp2)]
                print(f'result1: {cond}')
                break
        elif '<' in cond[j]:
            temp1, temp2 = cond[j].split("<")
            if result_dict[temp1] < int(temp2):
                cond[j] = [result_dict[temp1], '<', int(temp2)]
                print(f'result2: {cond}')
                break
        else:
            print(f'result3: {cond}')  
    if cond[j] in [[result_dict[temp1], '>', int(temp2)], [result_dict[temp1], '<', int(temp2)]]: 
        print(cond)
        if cond[-1] == 'A' or cond[-1] == 'R':
            total.append([result_dict, cond])
        break
    else:
        print(cond)
        if cond == 'A' or cond == 'R':
            total.append([result_dict, cond])


selected val: ['a>1716:R', 'A']
result3: ['a>1716', 'R']
['a>1716', 'R']
result3: ['A']
['A']


In [90]:
total

[]

In [72]:
print(result_dict)

{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}


## Solution 1

In [99]:
block1, block2 = open("input.txt").read().split("\n\n")

workflows = {}

for line in block1.splitlines():
    name, rest = line[:-1].split("{")
    rules = rest.split(",")
    workflows[name] = ([], rules.pop())
    for rule in rules:
        comparison, target = rule.split(":")
        key = comparison[0]
        cmp = comparison[1]
        n = int(comparison[2:])
        workflows[name][0].append((key, cmp, n, target))

ops = {
    ">": int.__gt__,
    "<": int.__lt__
}

def accept(item, name = "in"):
    if name == "R":
        return False
    if name == "A":
        return True

    rules, fallback = workflows[name]
    
    for key, cmp, n, target in rules:
        if ops[cmp](item[key], n):
            return accept(item, target)
    
    return accept(item, fallback)

total = 0

for line in block2.splitlines():
    item = {}
    for segment in line[1:-1].split(","):
        ch, n = segment.split("=")
        item[ch] = int(n)
    if accept(item):
        total += sum(item.values())

print(total)

'''solution from https://github.com/hyper-neutrino/advent-of-code/blob/main/2023/day19p1.py'''

342650


In [100]:
block1, _ = open('input.txt').read().split("\n\n")

workflows = {}

for line in block1.splitlines():
    name, rest = line[:-1].split("{")
    rules = rest.split(",")
    workflows[name] = ([], rules.pop())
    for rule in rules:
        comparison, target = rule.split(":")
        key = comparison[0]
        cmp = comparison[1]
        n = int(comparison[2:])
        workflows[name][0].append((key, cmp, n, target))

def count(ranges, name = "in"):
    if name == "R":
        return 0
    if name == "A":
        product = 1
        for lo, hi in ranges.values():
            product *= hi - lo + 1
        return product
    
    rules, fallback = workflows[name]

    total = 0

    for key, cmp, n, target in rules:
        lo, hi = ranges[key]
        if cmp == "<":
            T = (lo, min(n - 1, hi))
            F = (max(n, lo), hi)
        else:
            T = (max(n + 1, lo), hi)
            F = (lo, min(n, hi))
        if T[0] <= T[1]:
            copy = dict(ranges)
            copy[key] = T
            total += count(copy, target)
        if F[0] <= F[1]:
            ranges = dict(ranges)
            ranges[key] = F
        else:
            break
    else:
        total += count(ranges, fallback)
            
    return total

print(count({key: (1, 4000) for key in "xmas"}))

'''solution from https://github.com/hyper-neutrino/advent-of-code/blob/main/2023/day19p2.py'''

130303473508222


'solution from https://github.com/hyper-neutrino/advent-of-code/blob/main/2023/day19p2.py'